In [1]:
from __future__ import print_function
import argparse

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [3]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [4]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    return test_loss

In [ ]:
use_cuda = torch.cuda.is_available()

torch.manual_seed(1)
device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=64, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=1000, shuffle=True, **kwargs)


model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

test_loss_list = []
for epoch in range(1, 11):
    train(model, device, train_loader, optimizer, epoch)
    test_loss = test(model, device, test_loader)
    test_loss_list.append(test_loss)

100.1%

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz


113.5%

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz


100.4%

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz


180.4%

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.300039
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.437761
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.365868
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.324973
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.224680
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.340386
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.063599
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.116107
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.146479
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.117738

Test set: Average loss: 0.1015, Accuracy: 9668/10000 (97%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.145585
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.101984
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.058953
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.063877
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.120996
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.034490
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.034536
Train Epoch: 2 [44800/60000

In [6]:
import numpy as np
from matplotlib import pyplot as plt
plt.figure()
plt.title('test loss')
plt.plot(np.arange(len(test_loss_list)), np.array(test_loss_list))

In [7]:
import random
rand = random.randint(0, 9989)

model.eval()
image_list = []
target_list = []
pred_list = []
with torch.no_grad():
    for idx, (data, target) in enumerate(test_loader):
        if idx>=rand and idx<rand+10:
            data, target = data.to(device), target.to(device)
            image_list.append(data.squeeze(0).squeeze(0).cpu().numpy())
            target_list.append(target.item())
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            pred_list.append(pred.item())
        else:
            continue

In [8]:
Row = 2
Column = 5
for idx, img in enumerate(image_list):
    plt.subplot(Row, Column, idx+1)
    plt.title('Label = {}\nPredict = {}'.format(target_list[idx], pred_list[idx]))
    plt.imshow(img, cmap='gray')
    plt.axis('off')